# Crear Target

## Ejercicio 1:
Cree el archivo `competencia_01.csv`, usando el `competencia_01_crudo.csv` con una variable adicional llamada `clase_ternaria`, que contenga las categorías **CONTINUA**, **BAJA+1** y **BAJA+2** 

### Ayudita 

Para practicar el muy útil y necesario lenguaje **SQL**, vamos a utilizar una base de datos **OLAP** llamada **DuckDB**. 

La documentación la puede encontrar [aquí](https://duckdb.org/docs/archive/0.8.1/sql/introduction)
Procedemos a instalarla, esto se debe ejecutar una sola vez

In [1]:
#%%bash 
#%pip install duckdb
#%pip install jupysql
#%pip install duckdb-engine

Configuramos el entorno de ejecución. Si ya tiene todo instalado, solo necesita ejecutar esta celda para empezar a usar **duckdb** 

In [1]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:default:

Y ya podemos usar **SQL** dentro de una notebook!

In [2]:
%%sql
select 'hola mundo' 

,'hola mundo'
0,hola mundo


Para cargar el archivo `.csv` a una tabla:

In [3]:
%%sql
create or replace table competencia_01 as 
select 
    *
from read_csv_auto('C:\Users\feder\Documents\Maestria_en_Ciencia_de_datos\4_DM_en_Economia_y_Finanzas\datasets/competencia_01_crudo.csv')

,Success


Hagamos unas queries básicas para comprobar que todo esta funcionando bien.

In [4]:
%sql select * from competencia_01 limit 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,29183981,202103,1,0,0,50,197,14468.81,125765.29,2389.82,...,0.0,7,0.00,-114954.00,0.00,5938,101050.66,68,0,71811.06
1,29184630,202103,1,0,0,59,322,11901.57,74158.93,18750.68,...,0.0,1,0.00,-40330.15,17.59,4089,26834.09,7,0,3894.36
2,29185433,202103,1,0,0,68,268,847.15,21672.47,481.62,...,0.0,21,4692.00,-1173.00,0.00,7829,1651.36,3,0,1560.09
3,29185587,202103,1,0,0,79,322,4976.94,47735.98,1839.31,...,0.0,1,0.00,-15988.67,0.00,7580,30025.29,11,0,1700.85
4,29185646,202103,1,0,0,60,257,2860.45,37800.71,4035.40,...,0.0,21,380616.14,-97383.25,0.00,7827,359610.70,31,0,15600.90


In [5]:
%%sql
select 
    foto_mes
    , count(*) as cantidad -- cuenta cuantos casos hay en cada foto_mes 
                           -- y lo guarda en un campo llamado cantidad
from competencia_01
group by foto_mes

,foto_mes,cantidad
0,202103,163324
1,202104,163637
2,202105,164102


In [53]:
%%sql 
with mes_1 as (
select numero_de_cliente
from competencia_01
where foto_mes = '202105')

select comp.numero_de_cliente,
comp.foto_mes,
c.numero_de_cliente
from competencia_01 comp
left join mes_1 c
    on comp.numero_de_cliente = c.numero_de_cliente
where c.numero_de_cliente is null

,numero_de_cliente,foto_mes,numero_de_cliente_2
0,29282120,202103,NaN
1,29358824,202103,NaN
2,29379652,202103,NaN
3,29413956,202103,NaN
4,29479114,202103,NaN
...,...,...,...
2976,181504495,202104,NaN
2977,182321662,202104,NaN
2978,182463626,202104,NaN
2979,185451295,202104,NaN


Perfecto, ahora cree una nueva tabla con la variable adicional que se le pide. Algunas funciones que le pueden ser útiles:  [where](https://duckdb.org/docs/sql/query_syntax/where), [left join](https://duckdb.org/docs/sql/query_syntax/from), [case statement](https://duckdb.org/docs/sql/expressions/case)



In [7]:
%%sql 
select numero_de_cliente,
foto_mes
from competencia_01
where numero_de_cliente in (29282120, 181504495, 29183981) -- primer caso baja +1, segundo caso baja +2
order by 2


RuntimeError: (duckdb.InvalidInputException) Invalid Input Error: Attempting to execute an unsuccessful or closed pending query result
Error: Invalid Error: Current transaction is aborted (please ROLLBACK)
[SQL: select numero_de_cliente,
foto_mes
from competencia_01
where numero_de_cliente in (29282120, 181504495, 29183981) -- primer caso baja +1, segundo caso baja +2
order by 2]
(Background on this error at: https://sqlalche.me/e/20/f405)
If you need help solving this issue, send us a message: https://ploomber.io/community


In [15]:
%%sql

create or replace temp table competencia_01_ternaria as 
    with clientes as (
        select 
            numero_de_cliente,
            foto_mes,
            last_day(strptime(foto_mes, '%Y%m') + INTERVAL 1 MONTH) as mes_mas_1,
            last_day(strptime(foto_mes, '%Y%m') + INTERVAL 2 MONTH) as mes_mas_2,
        from competencia_01
    ),

    mes_1 as (
        select 
            c.numero_de_cliente,
            c.foto_mes,
            c1.numero_de_cliente as cliente_mas_1
        from clientes c 
        left join competencia_01 c1
            on c.numero_de_cliente = c1.numero_de_cliente
            and c.mes_mas_1 = last_day(strptime(c1.foto_mes, '%Y%m'))
     ),

    mes_2 as (
        select 
            c.numero_de_cliente,
            c.foto_mes,
            c1.numero_de_cliente as cliente_mas_2
        from clientes c 
        left join competencia_01 c1
            on c.numero_de_cliente = c1.numero_de_cliente
            and c.mes_mas_2 = last_day(strptime(c1.foto_mes, '%Y%m'))
    ),

    max_mes as (
        select 
            strptime(MAX(foto_mes),'%Y%m') as max_foto
        from competencia_01
        --group by 1
    ),

    final as (
    select 
        c1.*,      
        case
            when 
                cliente_mas_1 is not null 
                and cliente_mas_2 is null
                and strptime(c1.foto_mes, '%Y%m') = m.max_foto - INTERVAL 2 MONTH
            then 'BAJA+2'
            when 
                cliente_mas_1 is null 
                and cliente_mas_2 is null
                and strptime(c1.foto_mes, '%Y%m') <= m.max_foto - INTERVAL 1 MONTH
            then 'BAJA+1'
            when 
                (strptime(c1.foto_mes, '%Y%m') <= m.max_foto - INTERVAL 1 MONTH and cliente_mas_1 is not null)
                or 
                cliente_mas_2 is not null 
            then 'CONTINUA'
            else null end clase_ternaria
    from competencia_01 c1
    left join mes_1 m1 
        on m1.cliente_mas_1 = c1.numero_de_cliente
        and c1.foto_mes = m1.foto_mes
    left join mes_2 m2 
        on m2.cliente_mas_2 = c1.numero_de_cliente
        and c1.foto_mes = m2.foto_mes
    cross join max_mes m
    )
    
    select 
       *
    from final 


,Success


In [18]:
%%sql
select clase_ternaria, count(*) 
from competencia_01_ternaria
where foto_mes = '202103'
group by 1

,clase_ternaria,count_star()
0,CONTINUA,161312
1,BAJA+2,963
2,BAJA+1,1049


In [17]:
%%sql
COPY competencia_01_ternaria TO 'C:\Users\feder\Documents\Maestria_en_Ciencia_de_datos\4_DM_en_Economia_y_Finanzas\datasets/competencia_01.csv' (FORMAT CSV, HEADER)

,Success


## Ejercicio 1.1

* ¿Cuál es la nominalidad de cada clase?
* ¿Cuál es la proporción del target?

In [ ]:
%%sql
-- introduzca sus queries

Para guardar a un **.csv** simplemente debe ejecutar la siguiente sentencia

In [ ]:
%%sql
COPY competencia_01 TO '/home/aleb/dmeyf23/datasets/competencia_01.csv' (FORMAT CSV, HEADER)

## Ejercicio 2 - Avanzado
Use ahora el archivo `ejercicio_target.csv` y calcule para todos los clientes en todos los periodos su **clase_ternaria** al mismo tiempo